In [1]:
import os
import sys
sys.path.append('../')
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup the Worker

In [2]:
project = "ammonia-inventories-310"
databases = ["ammonia-reduced", "ecoinvent-3.10-cutoff"]
methods = "('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')"

In [3]:
# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = r"C:\APPS\GAMS\win64\40.1\gams.exe"

In [4]:
pulpo_worker = pulpo.PulpoOptimizer(project, databases, methods, directory)
pulpo_worker.intervention_matrix="ecoinvent-3.10-biosphere"

In [5]:
pulpo_worker.get_lci_data()

### Get the Choices

In [6]:
choices_biomethane = [
    "biogas upgrading to biomethane, chemical scrubbing",
    "biogas upgrading to biomethane, chemical scrubbing w/ CCS",
    "biogas upgrading to biomethane, water scrubbing",
    "biogas upgrading to biomethane, water scrubbing w/ CCS"
]

choices_methane_market = [
    "market for methane",
    "market group for natural gas, high pressure"
]

choices_hydrogen = [
    "hydrogen production, steam methane reforming fg",
    "hydrogen production, steam methane reforming, w/ CCS",
    "hydrogen production, PEM electrolysis, yellow",
    "hydrogen production, plastics gasification",
    "hydrogen production, plastics gasification, w/ CCS"
]

choices_hydrogen_market = [
    "market for hydrogen",
    "market for hydrogen, gaseous, low pressure"
]

choices_heat = [
    "heat from hydrogen",
    "heat from methane, w/ CCS",
    "heat from methane",
]

choices_ammonia = [
    "ammonia production, steam methane reforming",
    "ammonia production, steam methane reforming, w/ CCS",
    "ammonia production, from nitrogen and hydrogen"
]

choices_ammonia_market = [
    "market for ammonia",
    "market for ammonia, anhydrous, liquid"
]



In [7]:
# Retrieve activities for each category and assign to appropriately named variables
biomethane_activities = pulpo_worker.retrieve_activities(activities=choices_biomethane, locations=["RER", "Europe without Switzerland"])
methane_market_activities = pulpo_worker.retrieve_activities(activities=choices_methane_market, locations=["RER", "Europe without Switzerland"])
hydrogen_activities = pulpo_worker.retrieve_activities(activities=choices_hydrogen, locations=["RER", "Europe without Switzerland"])
hydrogen_market_activities = pulpo_worker.retrieve_activities(activities=choices_hydrogen_market, locations=["RER", "Europe without Switzerland"])
heat_activities = pulpo_worker.retrieve_activities(activities=choices_heat, locations=["RER", "Europe without Switzerland"])
ammonia_activities = pulpo_worker.retrieve_activities(activities=choices_ammonia, locations=["RER", "Europe without Switzerland"])
ammonia_market_activities = pulpo_worker.retrieve_activities(activities=choices_ammonia_market, locations=["RER", "Europe without Switzerland"])

In [8]:
# Display the results
print("Biomethane Activities:", biomethane_activities)
print("Methane Market Activities:", methane_market_activities)
print("Hydrogen Activities:", hydrogen_activities)
print("Hydrogen Market Activities:", hydrogen_market_activities)
print("Heat Activities:", heat_activities)
print("Ammonia Activities:", ammonia_activities)
print("Ammonia Market Activities:", ammonia_market_activities)

Biomethane Activities: ['biogas upgrading to biomethane, water scrubbing' (cubic meter, RER, None), 'biogas upgrading to biomethane, chemical scrubbing' (cubic meter, RER, None), 'biogas upgrading to biomethane, chemical scrubbing w/ CCS' (cubic meter, RER, None), 'biogas upgrading to biomethane, water scrubbing w/ CCS' (cubic meter, RER, None)]
Methane Market Activities: ['market for methane' (cubic meter, RER, None), 'market group for natural gas, high pressure' (cubic meter, Europe without Switzerland, None)]
Hydrogen Activities: ['hydrogen production, PEM electrolysis, yellow' (kilogram, RER, None), 'hydrogen production, plastics gasification' (kilogram, RER, None), 'hydrogen production, plastics gasification, w/ CCS' (kilogram, RER, None), 'hydrogen production, steam methane reforming fg' (kilogram, RER, None), 'hydrogen production, steam methane reforming, w/ CCS' (kilogram, RER, None)]
Hydrogen Market Activities: ['market for hydrogen' (kilogram, RER, None), 'market for hydrogen

In [9]:
choices = {
    "hydrogen": {x: 1e10 for x in hydrogen_activities},
    "heat": {x: 1e10 for x in heat_activities},
    "biomethane": {x: 1e10 for x in biomethane_activities},
    "ammonia": {x: 1e10 for x in ammonia_activities},
    "methane_market": {x: 1e10 for x in methane_market_activities},
    "hydrogen_market": {x: 1e10 for x in hydrogen_market_activities},
    "ammonia_market": {x: 1e10 for x in ammonia_market_activities},
}


### Demand

In [10]:
ammonia_market = pulpo_worker.retrieve_activities(activities="market for ammonia")

In [13]:
demand = {"ammonia_market": 1}

### Instantiate and Solve

In [14]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


In [15]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\APPS\GAMS\win64\40.1\gams.exe

GAMS WORKING DIRECTORY: C:\Users\FLECHT~1\AppData\Local\Temp\tmp68ckiw6f



In [17]:
pulpo_worker.save_results(choices=choices, demand=demand, name='reduced_ammonia_showcase.xlsx')

In [16]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)


The following demand / functional unit has been specified:


,demand
ammonia_market,1


No additional constraints have been passed.

Impacts contained in the objective:


,Key,Value
0,"('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')",-1.739579



The following choices were made:


hydrogen  \
                                                                                                                   Process   
Process 0               hydrogen production, PEM electrolysis, yellow | hydrogen production, PEM electrolysis, green | RER   
Process 1                    hydrogen production, plastics gasification | hydrogen production, plastics gasification | RER   
Process 2  hydrogen production, plastics gasification, w/ CCS | hydrogen production, plastics gasification, with CCS | RER   
Process 3                          hydrogen production, steam methane reforming fg | hydrogen production, biomethane | RER   
Process 4           hydrogen production, steam methane reforming, w/ CCS | hydrogen production, biomethane, with CCS | RER   

                                   \
               Capacity     Value   
Process 0  1.000000e+10  0.000000   
Process 1  1.000000e+10  0.000000   
Process 2  1.000000e+10  0.000000   
Process 3  1.000000e+10  0.000000   
Process 4  1.000000e+10  0.122404   

                                                                       heat  \
                                                                    Process   
Process 0  heat from methane, w/ CCS | heat from biomethane, with CCS | RER   
Process 1                       heat from methane | heat from methane | RER   
Process 2                     heat from hydrogen | heat from hydrogen | RER   
Process 3                                                               NaN   
Process 4                                                               NaN   

                                   \
               Capacity     Value   
Process 0  1.000000e+10   0.00000   
Process 1  1.000000e+10   0.00000   
Process 2  1.000000e+10  13.95406   
Process 3           NaN       NaN   
Process 4           NaN       NaN   

                                                                                     biomethane  \
                                                                                        Process   
Process 0            biogas upgrading to biomethane, water scrubbing | biomethane, 24 bar | RER   
Process 1         biogas upgrading to biomethane, chemical scrubbing | biomethane, 24 bar | RER   
Process 2  biogas upgrading to biomethane, chemical scrubbing w/ CCS | biomethane, 24 bar | RER   
Process 3     biogas upgrading to biomethane, water scrubbing w/ CCS | biomethane, 24 bar | RER   
Process 4                                                                                   NaN   

                                   \
               Capacity     Value   
Process 0  1.000000e+10  0.000000   
Process 1  1.000000e+10  0.000000   
Process 2  1.000000e+10  1.157785   
Process 3  1.000000e+10  0.000000   
Process 4           NaN       NaN   

                                                                                                         ammonia  \
                                                                                                         Process   
Process 0  ammonia production, from nitrogen and hydrogen | ammonia production, from nitrogen and hydrogen | RER   
Process 1   ammonia production, steam methane reforming, w/ CCS | ammonia production, biomethane, with CCS | RER   
Process 2                     ammonia production, steam methane reforming | ammonia production, biomethane | RER   
Process 3                                                                                                    NaN   
Process 4                                                                                                    NaN   

           ...            \
           ...     Value   
Process 0  ...  0.000000   
Process 1  ...  1.000141   
Process 2  ...  0.000000   
Process 3  ...       NaN   
Process 4  ...       NaN   

                                                                                                  methane_market  \
                                                                             